In [1]:
import sys

sys.path.append('/home/max/myrepos/vlstm_cuda/')

import torch
import triton
import triton.language as tl

In [2]:
triton.runtime.driver.active.get_current_target()

GPUTarget(backend='cuda', arch=89, warp_size=32)

In [3]:
from src_triton.mlstm_parallel_fw_v0.mlstm import mlstm_fw

In [4]:
S = 64 #32 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 64 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [5]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(1) #TODO from here: with seed=0 even the pytorch version alone breaks for float16 and bfloat16
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# qs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# # vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 100.
# # vs = torch.zeros((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs[:,:,1,0] = 7.
# qs[:,:,1,0] = 1.

# vs[:,:,1,16] = 8.
# qs[:,:,1,16] = 1.
# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [6]:
# inputs float16
dtype_fp16 = torch.float16
qs_half = qs.to(dtype=dtype_fp16)
ks_half = ks.to(dtype=dtype_fp16)
vs_half = vs.to(dtype=dtype_fp16)
igs_half = igs.to(dtype=dtype_fp16)
fgs_half = fgs.to(dtype=dtype_fp16)
dHs_half = dHs.to(dtype=dtype_fp16)

In [7]:
# inputs bfloat16
dtype_bf16 = torch.bfloat16
qs_bf16 = qs.to(dtype=dtype_bf16)
ks_bf16 = ks.to(dtype=dtype_bf16)
vs_bf16 = vs.to(dtype=dtype_bf16)
igs_bf16 = igs.to(dtype=dtype_bf16)
fgs_bf16 = fgs.to(dtype=dtype_bf16)
dHs_bf16 = dHs.to(dtype=dtype_bf16)

In [8]:
hs_half, m_half, n_half = mlstm_fw(matQ=qs_half, matK=ks_half, matV=vs_half, vecI=igs_half, vecF=fgs_half)

Triton grid: (1, 1, 1), BLOCK_Q: 64, BLOCK_KV: 32


In [9]:
qs_half, hs_half

(tensor([[[[-0.2964,  2.6758, -0.1409,  ..., -0.8525,  0.3560, -1.3682],
           [-1.4365, -1.5195,  0.2291,  ...,  0.4343,  0.7642, -1.0527],
           [-0.8398,  0.4783,  0.6865,  ..., -0.5474, -0.5615,  0.0093],
           ...,
           [ 0.2793,  1.0586, -0.6704,  ..., -0.3682,  0.5264,  0.2561],
           [-1.5283,  0.8374,  0.0970,  ..., -0.6860,  0.3206,  0.7261],
           [-1.3105,  1.0488, -0.1710,  ..., -0.2256,  0.2086, -0.1392]]]],
        device='cuda:0', dtype=torch.float16),
 tensor([[[[-0.2964,  2.6758, -0.1409,  ..., -0.8525,  0.3560, -1.3682],
           [-1.4365, -1.5195,  0.2291,  ...,  0.4343,  0.7642, -1.0527],
           [-0.8398,  0.4783,  0.6865,  ..., -0.5474, -0.5615,  0.0093],
           ...,
           [ 0.2793,  1.0586, -0.6704,  ..., -0.3682,  0.5264,  0.2561],
           [-1.5283,  0.8374,  0.0970,  ..., -0.6860,  0.3206,  0.7261],
           [-1.3105,  1.0488, -0.1710,  ..., -0.2256,  0.2086, -0.1392]]]],
        device='cuda:0', dtype=torch.fl

In [10]:
torch.allclose(hs_half, qs_half)

True

In [11]:
qs_half.shape

torch.Size([1, 1, 64, 64])

In [12]:
qs_half.stride(0), qs_half.stride(1), qs_half.stride(2), qs_half.stride(3)

(4096, 4096, 64, 1)

In [13]:
3 & 1

1

In [14]:
bool(3 & 2)

True